In [21]:
import pandas as pd
from tqdm import trange, tqdm
import numpy as np
import torch
import sys
from collections import defaultdict
sys.path.append('/mnt/nfs/zhangtl/utils/')
from util import myout
import json
import pickle as pkl

import dgl

## embeds

In [9]:
subreddits = pd.read_csv('../../raw_data/reddit/web-redditEmbeddings-subreddits.csv', header=None)
subreddits

,0,1,2,3,4,5,6,7,8,9,...,291,292,293,294,295,296,297,298,299,300
0,spiders,0.158972,0.285813,0.226329,-0.183338,-0.163159,-0.188910,0.422705,0.168602,0.178193,...,-0.025041,0.184655,0.111942,-1.125799,0.223129,-0.387323,-0.234882,-0.252297,-0.054315,0.500009
1,askreddit,-0.499114,0.323983,-0.424809,-0.222705,-0.327201,0.412856,1.040088,0.996697,-1.256240,...,0.072151,0.226658,0.535705,0.039861,-0.027087,-0.239952,0.554510,0.676439,-0.328846,0.941547
2,globaloffensivetrade,-0.023145,-1.199374,1.661484,-1.025296,1.424670,-1.179507,-1.035703,0.329301,-0.567272,...,2.998430,-1.235703,-0.326850,4.381238,-1.965378,-2.369674,-0.615389,2.010601,-0.919314,-2.149625
3,fireteams,2.492506,-2.529917,-0.448484,-3.543441,-0.586122,-0.101517,1.837567,-0.215715,-2.460388,...,-1.875214,0.969291,0.497958,2.941132,2.423561,1.315691,-0.959618,0.151845,-0.852492,-0.467526
4,funny,-0.819370,-0.865261,0.301753,0.018787,0.201029,0.070462,0.903895,1.038460,-0.306950,...,1.364194,0.830233,-0.137822,-2.204667,-0.162008,0.390291,-0.556404,0.322684,0.641634,1.320716
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51273,33rd,-0.000076,-0.001643,-0.002896,0.000581,-0.003509,-0.000827,0.002095,0.000123,0.004370,...,-0.001570,-0.002246,0.003491,-0.004310,0.001598,0.005168,-0.001156,0.001575,-0.001938,0.001594
51274,bestofvic2015,-0.000203,0.012698,-0.007575,0.006486,-0.001693,0.004743,-0.002401,0.000121,0.001600,...,0.001392,0.001258,0.002970,-0.014106,-0.002713,-0.008648,0.003660,-0.005755,-0.001790,-0.001313
51275,aberystwyth,-0.001563,0.004733,-0.000082,-0.000982,-0.001511,0.001285,0.002187,0.002967,0.000660,...,0.002213,0.000520,-0.000564,-0.010018,0.002507,-0.000651,0.004169,-0.004672,0.002371,0.001073
51276,mail_forwarding,0.009269,0.024779,-0.017018,-0.007228,0.002351,0.003462,-0.008446,0.004544,0.022505,...,-0.002591,0.003482,0.001042,-0.002796,0.010739,-0.009148,0.000363,-0.003144,0.001945,-0.017272


In [10]:
sub2idx = {sub:idx for idx,sub in enumerate(subreddits[0].unique().tolist())}
sub_embs = subreddits.iloc[:, 1:].values
myout(sub2idx, sub_embs)

sub2idx : len=51278, dict([spiders: 0, askreddit: 1, globaloffensivetrade: 2, fireteams: 3, funny: 4, the_donald: 5, ...])
sub_embs : shape=(51278, 300)
[[ 1.589720e-01  2.858130e-01  2.263290e-01 ... -2.522970e-01
  -5.431500e-02  5.000090e-01]
 [-4.991140e-01  3.239830e-01 -4.248090e-01 ...  6.764390e-01
  -3.288460e-01  9.415470e-01]
 [-2.314500e-02 -1.199374e+00  1.661484e+00 ...  2.010601e+00
  -9.193140e-01 -2.149625e+00]
 ...
 [-1.563000e-03  4.733000e-03 -8.200000e-05 ... -4.672000e-03
   2.371000e-03  1.073000e-03]
 [ 9.269000e-03  2.477900e-02 -1.701800e-02 ... -3.144000e-03
   1.945000e-03 -1.727200e-02]
 [ 4.570000e-03  1.240300e-02 -3.630000e-04 ... -2.030000e-04
   3.747000e-03  3.650000e-04]]


## load

In [11]:
papers = pd.read_csv('../../raw_data/reddit/soc-redditHyperlinks-title.tsv', sep='\t')
# papers.columns = ['source', 'target', 'rating', 'time']
# papers = papers.sort_values(by=['time', 'source', 'target'])
papers = papers.sort_values(by=['TIMESTAMP', 'SOURCE_SUBREDDIT', 'TARGET_SUBREDDIT'])
papers

,SOURCE_SUBREDDIT,TARGET_SUBREDDIT,POST_ID,TIMESTAMP,LINK_SENTIMENT,PROPERTIES
23,agitation,changemyview,1u5cj7s,2013-12-31 16:20:20,1,"102.0,91.0,0.843137254902,0.0,0.0882352941176,..."
0,rddtgaming,rddtrust,1u4pzzs,2013-12-31 16:39:18,1,"25.0,23.0,0.76,0.0,0.44,0.12,0.12,4.0,4.0,0.0,..."
35,examplesofevil,pics,1u5lhas,2013-12-31 16:40:29,1,"96.0,83.0,0.833333333333,0.0,0.0625,0.14583333..."
26,languagelearning,linguistics,1u5dkls,2013-12-31 17:41:16,1,"132.0,115.0,0.810606060606,0.0,0.0530303030303..."
1,xboxone,battlefield_4,1u4tmfs,2013-12-31 17:59:11,1,"100.0,88.0,0.78,0.02,0.08,0.13,0.07,16.0,16.0,..."
...,...,...,...,...,...,...
571891,peoplewhosayheck,sex,68iffo,2017-04-30 16:36:07,-1,"116.0,100.0,0.724137931034,0.0172413793103,0.1..."
571923,peoplewhosayheck,spiderman,68iqh6s,2017-04-30 16:36:07,1,"121.0,105.0,0.719008264463,0.0165289256198,0.0..."
571902,peoplewhosayheck,teenmom,68ii97s,2017-04-30 16:36:07,1,"137.0,115.0,0.759124087591,0.0,0.0510948905109..."
571892,brasilonreddit,bad_cop_no_donut,68ifuvs,2017-04-30 16:47:50,1,"194.0,162.0,0.773195876289,0.0,0.0670103092784..."


In [12]:
papers['time'] = papers['TIMESTAMP'].apply(lambda u: int(''.join(u.split(' ')[0].split('-'))))
papers = papers.sort_values(by=['time', 'SOURCE_SUBREDDIT', 'TARGET_SUBREDDIT'])
papers

,SOURCE_SUBREDDIT,TARGET_SUBREDDIT,POST_ID,TIMESTAMP,LINK_SENTIMENT,PROPERTIES,time
23,agitation,changemyview,1u5cj7s,2013-12-31 16:20:20,1,"102.0,91.0,0.843137254902,0.0,0.0882352941176,...",20131231
9,bestof,confession,1u56b3s,2013-12-31 21:41:46,1,"41.0,37.0,0.80487804878,0.0,0.121951219512,0.1...",20131231
15,bitcoinmining,bitcoin,1u58kus,2013-12-31 23:17:46,1,"82.0,70.0,0.756097560976,0.0,0.268292682927,0....",20131231
5,cancer,fuckcancer,1u5216s,2013-12-31 18:59:26,1,"94.0,80.0,0.776595744681,0.0,0.106382978723,0....",20131231
228,circlejerkaustralia,australia,1u7vo8s,2013-12-31 18:42:38,1,"102.0,90.0,0.823529411765,0.0,0.598039215686,0...",20131231
...,...,...,...,...,...,...,...
566524,winniethepooh,art,671bobs,2017-04-30 08:32:26,1,"76.0,66.0,0.842105263158,0.0,0.0921052631579,0...",20170430
571706,womeninred,pics,68grmys,2017-04-30 10:21:17,1,"91.0,76.0,0.703296703297,0.0,0.0989010989011,0...",20170430
571606,worldofwarships,warships,68frpzs,2017-04-30 07:26:04,1,"43.0,38.0,0.767441860465,0.0,0.162790697674,0....",20170430
571640,xbox_jp,xboxone,68g1xms,2017-04-30 08:40:29,1,"30.0,28.0,0.833333333333,0.0333333333333,0.3,0...",20170430


## cut

In [14]:
cut_step = len(papers) / 20
papers['new_time'] = (papers.index.values // cut_step).astype(np.int)
papers = papers.sort_values(by=['new_time', 'time', 'SOURCE_SUBREDDIT', 'TARGET_SUBREDDIT'])
papers

/tmp/ipykernel_640855/3604777765.py:2: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  papers['new_time'] = (papers.index.values // cut_step).astype(np.int)


,SOURCE_SUBREDDIT,TARGET_SUBREDDIT,POST_ID,TIMESTAMP,LINK_SENTIMENT,PROPERTIES,time,new_time
23,agitation,changemyview,1u5cj7s,2013-12-31 16:20:20,1,"102.0,91.0,0.843137254902,0.0,0.0882352941176,...",20131231,0
9,bestof,confession,1u56b3s,2013-12-31 21:41:46,1,"41.0,37.0,0.80487804878,0.0,0.121951219512,0.1...",20131231,0
15,bitcoinmining,bitcoin,1u58kus,2013-12-31 23:17:46,1,"82.0,70.0,0.756097560976,0.0,0.268292682927,0....",20131231,0
5,cancer,fuckcancer,1u5216s,2013-12-31 18:59:26,1,"94.0,80.0,0.776595744681,0.0,0.106382978723,0....",20131231,0
228,circlejerkaustralia,australia,1u7vo8s,2013-12-31 18:42:38,1,"102.0,90.0,0.823529411765,0.0,0.598039215686,0...",20131231,0
...,...,...,...,...,...,...,...,...
566524,winniethepooh,art,671bobs,2017-04-30 08:32:26,1,"76.0,66.0,0.842105263158,0.0,0.0921052631579,0...",20170430,19
571706,womeninred,pics,68grmys,2017-04-30 10:21:17,1,"91.0,76.0,0.703296703297,0.0,0.0989010989011,0...",20170430,19
571606,worldofwarships,warships,68frpzs,2017-04-30 07:26:04,1,"43.0,38.0,0.767441860465,0.0,0.162790697674,0....",20170430,19
571640,xbox_jp,xboxone,68g1xms,2017-04-30 08:40:29,1,"30.0,28.0,0.833333333333,0.0333333333333,0.3,0...",20170430,19


In [15]:
for ii in range(0, 20):
    print(ii, len(np.where(papers['new_time'].to_numpy()==ii)[0]))

0 28597
1 28596
2 28597
3 28596
4 28596
5 28597
6 28596
7 28596
8 28597
9 28596
10 28596
11 28597
12 28596
13 28596
14 28597
15 28596
16 28596
17 28597
18 28596
19 28596


## build graph

In [16]:
def update_idx(idx, dic, cnt, feats, feat_dim, no_emb, ff):
    if idx not in dic:
        dic[idx] = cnt
        cnt += 1
        # feats.append(torch.tensor(ff).to(torch.float32))
        if idx in sub2idx:
            feats.append(torch.tensor(sub_embs[sub2idx[idx]]).to(torch.float32))
        else:
            feats.append(torch.rand(feat_dim).to(torch.float32))
            no_emb += 1
    return dic, cnt, feats, no_emb

In [17]:
start_year, end_year = 0, 20
feat_dim = 300
index = {}

id2nid, cnt, no_emb = {}, 0, 0
lst, feats = [], []

for ii in trange(len(papers)):
    year = int(papers['new_time'].iloc[ii])
    source = str(papers['SOURCE_SUBREDDIT'].iloc[ii])
    target = str(papers['TARGET_SUBREDDIT'].iloc[ii])
    weight = int(papers['LINK_SENTIMENT'].iloc[ii])
    ff = [float(item) for item in papers['PROPERTIES'].iloc[ii].split(',')]
    
    id2nid, cnt, feats, no_emb = update_idx(source, id2nid, cnt, feats, feat_dim, no_emb, ff)
    id2nid, cnt, feats, no_emb = update_idx(target, id2nid, cnt, feats, feat_dim, no_emb, ff)
    
    lst.append((id2nid[source], id2nid[target], weight, year))

feat = torch.stack(feats)
src = torch.tensor([item[0] for item in lst])
tgt = torch.tensor([item[1] for item in lst])
rel = torch.tensor([item[2] for item in lst])
tsp = torch.tensor([item[3] for item in lst])

myout(feat, src, tgt, rel, id2nid, id2nid, no_emb)

100%|██████████| 571927/571927 [00:31<00:00, 17917.96it/s]


feat : shape=torch.Size([54075, 300])
tensor([[-0.1257, -0.0357,  0.0769,  ...,  0.1096,  0.1095, -0.1865],
        [-0.5796,  0.0761, -0.1174,  ..., -0.0368, -0.0934,  0.1401],
        [-0.5187,  0.1825,  0.2709,  ..., -0.0557,  0.2146,  0.6653],
        ...,
        [ 0.0377,  0.2591,  0.7070,  ...,  0.0453,  0.3718,  0.6333],
        [-0.0027, -0.0553,  0.0791,  ..., -0.0025,  0.0438,  0.0051],
        [ 0.1238,  0.3724,  0.2189,  ...,  0.7090,  0.5399,  0.8360]])
src : shape=torch.Size([571927]), tensor([    0,     2,     4,  ..., 22050, 50389,    48])
tgt : shape=torch.Size([571927]), tensor([    1,     3,     5,  ..., 14425,    48,   431])
rel : shape=torch.Size([571927]), tensor([1, 1, 1,  ..., 1, 1, 1])
id2nid : len=54075, dict([agitation: 0, changemyview: 1, bestof: 2, confession: 3, bitcoinmining: 4, bitcoin: 5, ...])
id2nid : len=54075, dict([agitation: 0, changemyview: 1, bestof: 2, confession: 3, bitcoinmining: 4, bitcoin: 5, ...])
no_emb = 26363


In [18]:
graph = dgl.graph((src, tgt), num_nodes=len(feat))
graph.ndata['feat'] = feat

nid2id = {vv: kk for kk, vv in id2nid.items()}
graph.ndata['raw_nid'] = torch.arange(len(feat))

graph.edata['rel'] = rel
graph.edata['ts'] = tsp
graph

Graph(num_nodes=54075, num_edges=571927,
      ndata_schemes={'feat': Scheme(shape=(300,), dtype=torch.float32), 'raw_nid': Scheme(shape=(), dtype=torch.int64)}
      edata_schemes={'rel': Scheme(shape=(), dtype=torch.int64), 'ts': Scheme(shape=(), dtype=torch.int64)})

In [19]:
dataset = 'rdt'
dgl.save_graphs(f'../data/{dataset}/graph.bin', [graph])

In [22]:
json.dump(id2nid, open(f'../data/{dataset}/id2nid.json', 'w'))

## gen cites

In [23]:
start_year, end_year = 0, 20
cites = {}
for year in range(start_year, end_year):
    cites[year] = defaultdict(int)

for ii in trange(len(papers)):
    year = int(papers['new_time'].iloc[ii])
    target = str(papers['TARGET_SUBREDDIT'].iloc[ii])
    cites[year][target] += 1
myout(cites[5])

100%|██████████| 571927/571927 [00:08<00:00, 69801.56it/s]

 : len=4520, dict([gamecollecting: 6, askreddit: 934, todayilearned: 493, sweden: 49, technologyprotips: 2, aww: 133, ...])


In [24]:
tsp = graph.edata['ts']
ts_vals, ts_cuts = np.unique(tsp.numpy(), return_index=True)
ts_cuts = list(ts_cuts) + [len(tsp.numpy())]

num_ts = len(ts_vals)
ts_infos = np.stack([ts_vals, ts_cuts[0:num_ts], ts_cuts[1:num_ts+1]]).transpose()
myout(ts_cuts, ts_vals, ts_infos)

ts_cuts : len=21, list([0, 28597, 57193, ..., 514735, 543331, 571927])
ts_vals : shape=(20,), [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19]
ts_infos : shape=(20, 3)
[[     0      0  28597]
 [     1  28597  57193]
 [     2  57193  85790]
 [     3  85790 114386]
 [     4 114386 142982]
 [     5 142982 171579]
 [     6 171579 200175]
 [     7 200175 228771]
 [     8 228771 257368]
 [     9 257368 285964]
 [    10 285964 314560]
 [    11 314560 343157]
 [    12 343157 371753]
 [    13 371753 400349]
 [    14 400349 428946]
 [    15 428946 457542]
 [    16 457542 486138]
 [    17 486138 514735]
 [    18 514735 543331]
 [    19 543331 571927]]


In [25]:
labels = {}
nid2id = {v:k for k,v in id2nid.items()}
for year in range(start_year, end_year):
    left, right = ts_infos[np.where(ts_infos[:, 0]==year)[0][0], 1:]
    nids = graph.edges()[0][left:right].unique().tolist()
    ids = [nid2id[nid] for nid in nids]
    
    pdf = pd.DataFrame({'id': ids, 'nid': nids})
    tbar = trange(year+1, end_year, desc=str(year))
    for yy in tbar:
        cdf = pd.DataFrame({'id': list(cites[yy].keys()), str(yy): list(cites[yy].values())})
        cdf[str(yy)] = cdf[str(yy)].astype('float32')
        
        pdf = pd.merge(pdf, cdf, how='left', on='id')
        tbar.set_postfix(year=year, pdf=len(pdf))
    pdf.fillna(0, inplace=True)
    labels[year] = pdf

18: 100%|██████████| 1/1 [00:00<00:00, 134.39it/s, pdf=6863, year=18]
19: 0it [00:00, ?it/s]


In [26]:
labels[5]

,id,nid,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,agitation,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
1,bestof,2,54.0,65.0,43.0,60.0,45.0,54.0,33.0,38.0,41.0,31.0,33.0,35.0,33.0,35.0
2,bitcoinmining,4,0.0,2.0,0.0,1.0,2.0,0.0,0.0,6.0,1.0,1.0,0.0,0.0,1.0,0.0
3,bitcoin,5,176.0,137.0,115.0,97.0,95.0,69.0,67.0,62.0,72.0,50.0,42.0,74.0,49.0,59.0
4,cancer,6,3.0,1.0,1.0,2.0,2.0,1.0,0.0,0.0,6.0,2.0,2.0,5.0,2.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6346,retiredshittygif,23229,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6347,theartofswearing,23230,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6348,cameroncanada,23231,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6349,titlegold,23232,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [27]:
pkl.dump(labels, open(f'../data/{dataset}/labels.pkl', 'wb'))

In [28]:
def cumulative_log(df):
    colsn = list(df.columns)
    for i in range(3, len(colsn)):
        df[colsn[i]] += df[colsn[i-1]]
    df.iloc[:, 2:] = np.log(df.iloc[:, 2:] + 1)
    return df

labels_cum_log = {}
for year in range(start_year, end_year-1):
    labels_cum_log[year] = cumulative_log(labels[year])
labels_cum_log[end_year-2] = labels[end_year-2]
print(len(labels_cum_log))
labels_cum_log[5]

19


,id,nid,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,agitation,0,0.693147,0.693147,0.693147,0.693147,0.693147,0.693147,0.693147,1.098612,1.098612,1.098612,1.098612,1.386294,1.386294,1.386294
1,bestof,2,4.007333,4.787492,5.093750,5.407172,5.590987,5.774551,5.872118,5.973810,6.073045,6.142037,6.210600,6.278522,6.338594,6.398595
2,bitcoinmining,4,0.000000,1.098612,1.098612,1.386294,1.791759,1.791759,1.791759,2.484907,2.564949,2.639057,2.639057,2.639057,2.708050,2.708050
3,bitcoin,5,5.176150,5.749393,6.061457,6.265301,6.431331,6.536692,6.629363,6.708084,6.792345,6.846943,6.890609,6.963190,7.008505,7.060476
4,cancer,6,1.386294,1.609438,1.791759,2.079442,2.302585,2.397895,2.397895,2.397895,2.833213,2.944439,3.044523,3.258096,3.332205,3.332205
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6346,retiredshittygif,23229,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6347,theartofswearing,23230,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6348,cameroncanada,23231,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6349,titlegold,23232,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [ ]:
pkl.dump(labels_cum_log, open(f'../data/{dataset}/labels_cum_log.pkl', 'wb'))